In [8]:
!git clean -df

Removing B1F4.in


Create a folder with input file

In [9]:
%cd ~/Paesani/potential_fitting

/home/azonca/Paesani/potential_fitting


In [10]:
mol = "B1F4_B1F4"

In [11]:
%mkdir $mol

In [12]:
%cd $mol

/home/azonca/Paesani/potential_fitting/B1F4_B1F4


In [13]:
root_folder, = !pwd

In [14]:
!python ../polynomial_generation/src/generate_input_poly.py $mol

In [15]:
%ls

B1F4_B1F4.in


In [9]:
%cp /home/azonca/Paesani/potential_fitting/polynomial_generation/tests/1B/bf4.in .

In [11]:
!cat bf4.in

add_molecule['B1F4']

add_variable['B',  'a', 'F1', 'a',  'x-intra-BF']
add_variable['B',  'a', 'F2', 'a',  'x-intra-BF']
add_variable['B',  'a', 'F3', 'a',  'x-intra-BF']
add_variable['B',  'a', 'F4', 'a',  'x-intra-BF']
add_variable['F1', 'a', 'F2', 'a',  'x-intra-FF']
add_variable['F1', 'a', 'F3', 'a',  'x-intra-FF']
add_variable['F1', 'a', 'F4', 'a',  'x-intra-FF']
add_variable['F2', 'a', 'F3', 'a',  'x-intra-FF']
add_variable['F2', 'a', 'F4', 'a',  'x-intra-FF']
add_variable['F3', 'a', 'F4', 'a',  'x-intra-FF']


In [12]:
%%file config_poly.ini

[polynomial_generation]
molecule_definition = A1B2_A1B2.in
polynomial_order = 4

Writing config_poly.ini


In [13]:
%%file config_fit.ini

[fitting]
charge = [1. 1. 1. 1. 1.]
polarizability = [1. 1. 1. 1. 1.]
polarizability_damping = [1. 1. 1. 1. 1.]
excluded_pairs = []
training_set_filename = bf4_training_set.xyz

Writing config_fit.ini


In [14]:
!ls

A1B2_A1B2.in  bf4.in  config_fit.ini  config_poly.ini


In [15]:
# have to run from the folder
config_filename = "config_poly.ini"

In [16]:
import configparser
import os

config = configparser.ConfigParser()
config.read(config_filename)

['config_poly.ini']

# Polynomial generation

### Define input file

### Create maple input files

In [17]:
order = config.getint("polynomial_generation", "polynomial_order")

In [18]:
root_folder

'/home/azonca/Paesani/potential_fitting/A1B2_A1B2'

In [19]:
input_file = os.path.join(root_folder, config.get("polynomial_generation", "molecule_definition"))

In [20]:
os.mkdir("polynomial_generation")

In [21]:
os.chdir("polynomial_generation")

In [22]:
potential_fitting_root_folder = "/home/azonca/Paesani/potential_fitting/"

In [23]:
%pwd

'/home/azonca/Paesani/potential_fitting/A1B2_A1B2/polynomial_generation'

In [24]:
!$potential_fitting_root_folder/polynomial_generation/src/poly-gen_mb-nrg.pl $order $input_file > poly_generation.log 

In [25]:
ls

poly-direct.cpp      poly-grd.maple  poly-nogrd.maple
poly_generation.log  poly-model.h    vars.cpp


### Run maple and transform output to C++

In [26]:
for f in ["poly-nogrd", "poly-grd"]:
    maple_file = f + ".maple"
    !maple $maple_file
    c_file = f + ".c"
    cpp_file = f + ".cpp"
    !$potential_fitting_root_folder/polynomial_generation/src/clean-maple-c.pl < $c_file > $cpp_file

    |\^/|     Maple 2016 (X86 64 LINUX)
._|\|   |/|_. Copyright (c) Maplesoft, a division of Waterloo Maple Inc. 2016
 \  MAPLE  /  All rights reserved. Maple is a trademark of
 <____ ____>  Waterloo Maple Inc.
      |       Type ? for help.
>     p[1] := x11+x15+x12+x14:
>     p[2] := x13+x08+x10+x09:
>     p[3] := x07:
> 
>     p[4] := x06*x11+x03*x11+x06*x12+x03*x12+x03*x15+x06*x15+x03*x14+x06*x14:
>     p[5] := x07*x08+x07*x13+x07*x10+x07*x09:
>     p[6] := x04*x10+x01*x09+x05*x13+x01*x08+x02*x09+x02*x08+x05*x10+x04*x13:
>     p[7] := x05*x14+x05*x11+x02*x11+x04*x15+x01*x14+x01*x15+x02*x12+x04*x12:
>     p[8] := x03*x07+x06*x07:
>     p[9] := x08*x13+x09*x10+x08*x10+x09*x13:
>     p[10] := x07*x12+x07*x11+x07*x14+x07*x15:
>     p[11] := x13*x15+x10*x12+x13*x14+x09*x12+x08*x11+x10*x11+x09*x15+x08*x14:
>     p[12] := x10*x15+x09*x14+x12*x13+x11*x13+x08*x12+x09*x11+x10*x14+x08*x15:
>     p[13] := x05*x07+x01*x07+x02*x07+x04*x07:
>     p[14] := x14*x14+x15*x15+x11*x11+x12*x12:
>     p[

# Fitting

In [27]:
os.chdir("..")

In [28]:
os.mkdir("fitting")

In [29]:
os.chdir("fitting")

### Setup fitting and compile

In [35]:
!$potential_fitting_root_folder/fitting/1B/get_codes/setup.ba ../polynomial_generation/poly-grd.cpp ../polynomial_generation/poly-nogrd.cpp ../polynomial_generation/poly-direct.cpp ../polynomial_generation/poly-model.h $input_file $order 0 .

/home/azonca/Paesani/potential_fitting/A1B2_A1B2/fitting/../templates/normal
A1B2
cp: cannot stat ‘/home/azonca/Paesani/potential_fitting/A1B2_A1B2/fitting/../templates/normal/*’: No such file or directory
sed: can't read eval-1b-pot.cpp: No such file or directory
sed: can't read eval-1b-pot-wgrads.cpp: No such file or directory
sed: can't read fit-1b-polynomials.cpp: No such file or directory
sed: can't read stuff.h: No such file or directory
sed: can't read stuff.cpp: No such file or directory
sed: can't read training-set.cpp: No such file or directory
sed: can't read training-set.h: No such file or directory
sed: can't read fit-utils.cpp: No such file or directory
sed: can't read fit-utils.h: No such file or directory
sed: can't read x1b-v1.cpp: No such file or directory
sed: can't read x1b-v1.h: No such file or directory
sed: can't read x1b-v1x.cpp: No such file or directory
sed: can't read x1b-v1x.h: No such file or directory
sed: can't read eval-1b-pot-wgrads.cpp: No such file or

### Run fitting

First make sure that the `.xyz` training set is available

In [31]:
%cd B1F4/normal/4

[Errno 2] No such file or directory: 'B1F4/normal/4'
/home/azonca/Paesani/potential_fitting/A1B2_A1B2


In [32]:
!ln -s  ~/Paesani/bf4_training_set.xyz .

In [33]:
!./fit-1b-polynomials *_training_set.xyz > fit-1b-polynomials.log

/usr/bin/sh: ./fit-1b-polynomials: No such file or directory


Check fitted output coefficients

In [34]:
!cat fit-x1b-v1.dat

cat: fit-x1b-v1.dat: No such file or directory
